In [ ]:
!pip install dbt-snowflake

In [3]:
!mkdir -p ../models/l10_staging
!mkdir -p ../models/l20_transform
!mkdir -p ../models/l30_mart
!mkdir -p ../models/tests

In [4]:
%%writefile ../dbt_project.yml
name: 'snowflake_workshop'
version: '1.0.0'
config-version: 2
profile: 'default'
model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]
target-path: "target"
clean-targets:
  - "target"
  - "dbt_packages"
models:
  snowflake_workshop:
      # Applies to all files under models/example/
      example:
          materialized: view
          +enabled: false
      l10_staging:
          schema: l10_staging
          materialized: view
      l20_transform:
          schema: l20_transform
          materialized: view
      l30_mart:
          schema: l30_mart
          materialized: view

Overwriting ../dbt_project.yml


## Setup Database Connections

In [ ]:
%%writefile ../profiles.yml
default:
  outputs:
    dev:
      account: <>.us-east-1
      database: dbtKnoema
      password: <>
      role: accountadmin
      schema: dev
      threads: 1
      type: snowflake
      user: sprsag
      warehouse: compute_wh
    prod:
      account: <>.us-east-1
      database: dbtKnoema
      password: <>
      role: accountadmin
      schema: prod
      threads: 1
      type: snowflake
      user: sprsag
      warehouse: compute_wh
  target: dev

By default, dbt is generating a schema name by appending it to the target schema environment name(dev, prod). In this lab we are going to show you a quick way to override this macro, making our schema names to look exactly the same between dev and prod databases. For this, let's create a file macros\call_me_anything_you_want.sql with the following content:

In [5]:
%%writefile ../macros/call_me_anything_you_want.sql
{% macro generate_schema_name(custom_schema_name, node) -%}
    {%- set default_schema = target.schema -%}
    {%- if custom_schema_name is none -%}
        {{ default_schema }}
    {%- else -%}
        {{ custom_schema_name | trim }}
    {%- endif -%}
{%- endmacro %}


{% macro set_query_tag() -%}
  {% set new_query_tag = model.name %} {# always use model name #}
  {% if new_query_tag %}
    {% set original_query_tag = get_current_query_tag() %}
    {{ log("Setting query_tag to '" ~ new_query_tag ~ "'. Will reset to '" ~ original_query_tag ~ "' after materialization.") }}
    {% do run_query("alter session set query_tag = '{}'".format(new_query_tag)) %}
    {{ return(original_query_tag)}}
  {% endif %}
  {{ return(none)}}
{% endmacro %}

Writing ../macros/call_me_anything_you_want.sql


In [ ]:
%%writefile ../packages.yml
packages:
  - package: dbt-labs/dbt_utils
    version: 0.8.2

In [6]:
!dbt deps --profiles-dir .. --project-dir ..

15:47:23  Running with dbt=1.2.0
15:47:24  Installing dbt-labs/dbt_utils
15:47:25    Installed from version 0.8.4
15:47:25    Updated version available: 0.8.6
15:47:25  
15:47:25  Updates available for packages: ['dbt-labs/dbt_utils']                 
Update your versions in packages.yml, then run dbt deps


In [ ]:
!dbt debug  --profiles-dir .. --project-dir ..

In [9]:
!dbt seed  --profiles-dir .. --project-dir ..

15:58:53  Running with dbt=1.2.0
15:58:54  Unable to do partial parsing because profile has changed
15:58:54  Unable to do partial parsing because a project config has changed
15:58:57  [WARNING]: Test 'test.snowflake_workshop.fct_orders_negative_discount_amount' (tests/fct_orders_negative_discount_amount.sql) depends on a node named 'fct_orders' which was not found
15:58:57  Found 15 models, 6 tests, 0 snapshots, 0 analyses, 482 macros, 0 operations, 2 seed files, 2 sources, 0 exposures, 0 metrics
15:58:57  
15:59:16  Concurrency: 1 threads (target='dev')
15:59:16  
15:59:16  1 of 2 START seed file dev.manual_book1 ........................................ [RUN]
15:59:23  1 of 2 OK loaded seed file dev.manual_book1 .................................... [INSERT 7 in 6.78s]
15:59:23  2 of 2 START seed file dev.manual_book2 ........................................ [RUN]
15:59:30  2 of 2 OK loaded seed file dev.manual_book2 .................................... [INSERT 5 in 6.23s]
15:59:30  

In [15]:
!dbt run  --profiles-dir .. --project-dir ..

16:16:10  Running with dbt=1.2.0
16:16:10  Found 15 models, 6 tests, 0 snapshots, 0 analyses, 482 macros, 0 operations, 2 seed files, 2 sources, 0 exposures, 0 metrics
16:16:10  
16:16:34  Concurrency: 1 threads (target='dev')
16:16:34  
16:16:34  1 of 15 START view model l10_staging.base_knoema_fx_rates ...................... [RUN]
16:16:39  1 of 15 OK created view model l10_staging.base_knoema_fx_rates ................. [SUCCESS 1 in 4.90s]
16:16:39  2 of 15 START view model l10_staging.base_knoema_stock_history ................. [RUN]
16:16:43  2 of 15 OK created view model l10_staging.base_knoema_stock_history ............ [SUCCESS 1 in 4.70s]
16:16:43  3 of 15 START view model l20_transform.tfm_book ................................ [RUN]
16:16:49  3 of 15 OK created view model l20_transform.tfm_book ........................... [SUCCESS 1 in 5.32s]
16:16:49  4 of 15 START table model l20_transform.tfm_fx_rates ........................... [RUN]
16:16:59  4 of 15 OK created table mod

In [16]:
!dbt test  --profiles-dir .. --project-dir ..

16:18:23  Running with dbt=1.2.0
16:18:24  Found 15 models, 6 tests, 0 snapshots, 0 analyses, 482 macros, 0 operations, 2 seed files, 2 sources, 0 exposures, 0 metrics
16:18:24  
16:18:39  Concurrency: 1 threads (target='dev')
16:18:39  
16:18:39  1 of 6 START test not_null_tfm_book_instrument ................................. [RUN]
16:18:42  1 of 6 PASS not_null_tfm_book_instrument ....................................... [PASS in 3.27s]
16:18:42  2 of 6 START test not_null_tfm_fx_rates_currency_date .......................... [RUN]
16:18:46  2 of 6 PASS not_null_tfm_fx_rates_currency_date ................................ [PASS in 3.69s]
16:18:46  3 of 6 START test not_null_tfm_stock_history_company_symbol_date_stock_exchange_name  [RUN]
16:19:12  3 of 6 PASS not_null_tfm_stock_history_company_symbol_date_stock_exchange_name . [PASS in 26.60s]
16:19:12  4 of 6 START test relationships_tfm_book_instrument__company_symbol__ref_tfm_stock_history_  [RUN]
16:19:17  4 of 6 PASS relationships

In [ ]:
!dbt docs generate  --profiles-dir .. --project-dir ..

In [18]:
!dbt docs serve --profiles-dir .. --project-dir ..

16:21:33  Running with dbt=1.2.0
16:21:33  Serving docs at 0.0.0.0:8080
16:21:33  To access from your browser, navigate to:  http://localhost:8080
16:21:33  
16:21:33  
16:21:33  Press Ctrl+C to exit.
127.0.0.1 - - [05/Aug/2022 21:51:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2022 21:51:35] "GET /manifest.json?cb=1659716495450 HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2022 21:51:35] "GET /catalog.json?cb=1659716495450 HTTP/1.1" 200 -
^C
